In [1]:
import os, cv2, random
import numpy as np
from collections import defaultdict

# -------------------- CONFIG --------------------
image_dir = r"C:\Users\Projects\Age_Gender_classifier\images"
img_size  = 64

# -------------------- LOAD ----------------------
X, age_labels, gender_labels, race_labels = [], [], [], []

for fn in os.listdir(image_dir):
    if fn.endswith(".jpg"):
        parts = fn.split("_")
        if len(parts) >= 4:
            try:
                age, gender, race = map(int, parts[:3])
                img = cv2.resize(cv2.imread(os.path.join(image_dir, fn)),
                                 (img_size, img_size)).astype("float32") / 255.0
                X.append(img)
                age_labels.append(age)
                gender_labels.append([1,0] if gender == 0 else [0,1])
                race_labels.append(race)
            except Exception as e:
                print("❌", fn, e)
        else:
            print("⚠️ malformed:", fn)

X             = np.array(X)
age_labels    = np.array(age_labels)
gender_labels = np.array(gender_labels)
race_labels   = np.array(race_labels)

print("✅ Loaded:", X.shape, age_labels.shape, gender_labels.shape, race_labels.shape)


⚠️ malformed: 39_1_20170116174525125.jpg
❌ 53__0_20170116184028385.jpg invalid literal for int() with base 10: ''
⚠️ malformed: 61_1_20170109142408075.jpg
⚠️ malformed: 61_3_20170109150557335.jpg
✅ Loaded: (24102, 64, 64, 3) (24102,) (24102, 2) (24102,)


In [2]:
assert len(X) == len(age_labels) == len(gender_labels) == len(race_labels)

groups = defaultdict(list)
for idx, (age, g_onehot, race) in enumerate(zip(age_labels, gender_labels, race_labels)):
    groups[(age, int(np.argmax(g_onehot)), race)].append(idx)

random.seed(42)
train_idx, test_idx = [], []
for idx_list in groups.values():
    random.shuffle(idx_list)
    if len(idx_list) == 1:
        train_idx += idx_list
    elif len(idx_list) == 2:
        train_idx += [idx_list[0]]
        test_idx  += [idx_list[1]]
    else:
        k = int(round(0.8 * len(idx_list)))
        train_idx += idx_list[:k]
        test_idx  += idx_list[k:]

# slice
X_train, X_test         = X[train_idx],         X[test_idx]
age_train, age_test     = age_labels[train_idx].reshape(-1,1), age_labels[test_idx].reshape(-1,1)
gender_train, gender_test = gender_labels[train_idx],          gender_labels[test_idx]

print(f"Train: {len(train_idx)}  Test: {len(test_idx)}  Total: {len(X)}")


Train: 19241  Test: 4861  Total: 24102


In [3]:
# flatten images
X_train_flat = X_train.reshape(len(X_train), -1).astype("float32")
X_test_flat  = X_test.reshape(len(X_test),  -1).astype("float32")

# scale ages
MAX_AGE          = age_train.max()
age_train_scaled = age_train / MAX_AGE
age_test_scaled  = age_test  / MAX_AGE


In [4]:
def he_init(fan_in, fan_out):
    return np.random.randn(fan_in, fan_out) * np.sqrt(2.0 / fan_in)

def relu(x):              return np.maximum(0., x)
def relu_grad(x):         return (x > 0.).astype(np.float32)

def softmax(z):
    e = np.exp(z - z.max(axis=1, keepdims=True))
    return e / e.sum(axis=1, keepdims=True)

def mse(pred, target):         return np.mean((pred - target) ** 2)
def mse_grad(pred, target):    return 2 * (pred - target) / target.shape[0]

def cross_entropy(p, y, eps=1e-12):
    p = np.clip(p, eps, 1-eps)
    return -np.mean(np.sum(y * np.log(p), axis=1))

def cross_entropy_grad(p, y):  return (p - y) / y.shape[0]


In [5]:
class Dense:
    def __init__(self, in_dim, out_dim):
        self.W = he_init(in_dim, out_dim)
        self.b = np.zeros((1, out_dim))

    def forward(self, X):
        self.X = X
        return X @ self.W + self.b

    def backward(self, dY):
        m = dY.shape[0]
        self.dW = (self.X.T @ dY) / m
        self.db = np.mean(dY, axis=0, keepdims=True)
        return dY @ self.W.T

    def step(self, lr):
        self.W -= lr * self.dW
        self.b -= lr * self.db


In [6]:
class MultiTaskNN:
    def __init__(self, input_dim, alpha=1.0, beta=5.0):
        self.alpha, self.beta = alpha, beta
        self.s1 = Dense(input_dim, 512);  self.s2 = Dense(512, 256)
        self.a1 = Dense(256, 64);         self.a2 = Dense(64, 1)
        self.g1 = Dense(256, 64);         self.g2 = Dense(64, 2)

    def forward(self, X):
        z1 = self.s1.forward(X); a1 = relu(z1)
        z2 = self.s2.forward(a1); a2 = relu(z2)

        za1 = self.a1.forward(a2); aa1 = relu(za1)
        age_pred = self.a2.forward(aa1)

        zg1 = self.g1.forward(a2); ag1 = relu(zg1)
        logits  = self.g2.forward(ag1)
        gender_pred = softmax(logits)
        return (z1,z2,za1,zg1,a1,a2,aa1,ag1,logits), age_pred, gender_pred

    def backward(self, cache, age_pred, gender_pred, age_true, gender_true, lr):
        z1,z2,za1,zg1,a1,a2,aa1,ag1,_ = cache
        d_age = self.alpha * mse_grad(age_pred, age_true)
        d_gen = self.beta  * cross_entropy_grad(gender_pred, gender_true)

        # age head
        d = self.a2.backward(d_age)
        d = relu_grad(za1) * d
        d = self.a1.backward(d)

        # gender head
        dg = self.g2.backward(d_gen)
        dg = relu_grad(zg1) * dg
        dg = self.g1.backward(dg)

        d_shared = d + dg
        d_shared = self.s2.backward(relu_grad(z2) * d_shared)
        _        = self.s1.backward(relu_grad(z1) * d_shared)

        for layer in (self.s1, self.s2, self.a1, self.a2, self.g1, self.g2):
            layer.step(lr)

    def train_batch(self, X, age_true, gender_true, lr):
        cache, age_pred, gender_pred = self.forward(X)
        loss_age = mse(age_pred, age_true)
        loss_gen = cross_entropy(gender_pred, gender_true)
        self.backward(cache, age_pred, gender_pred, age_true, gender_true, lr)
        return self.alpha*loss_age + self.beta*loss_gen, loss_age, loss_gen

    def predict(self, X):
        _, age_pred, gender_pred = self.forward(X)
        return age_pred, gender_pred


In [7]:
def train_model(model, X_flat, age_scaled, gender, max_age,
                epochs=25, batch=128, lr=1e-4):
    N = X_flat.shape[0]
    hist_loss, hist_mae, hist_acc = [], [], []
    for ep in range(1, epochs+1):
        perm = np.random.permutation(N)
        X_flat, age_scaled, gender = X_flat[perm], age_scaled[perm], gender[perm]

        tot_loss = 0
        for i in range(0, N, batch):
            xb = X_flat[i:i+batch]
            ab = age_scaled[i:i+batch]
            gb = gender[i:i+batch]
            loss, *_ = model.train_batch(xb, ab, gb, lr)
            tot_loss += loss * xb.shape[0]

        age_pred_sc, g_pred = model.predict(X_flat)
        age_pred = age_pred_sc.squeeze() * max_age
        mae  = np.mean(np.abs(age_pred - (age_scaled.squeeze()*max_age)))
        acc  = np.mean(np.argmax(g_pred,1) == np.argmax(gender,1))

        hist_loss.append(tot_loss/N); hist_mae.append(mae); hist_acc.append(acc)
        print(f"Epoch {ep:02d} | Loss {tot_loss/N:.4f} | MAE {mae:.2f} | Acc {acc*100:.2f}%")
    return hist_loss, hist_mae, hist_acc

from sklearn.metrics import mean_absolute_error, accuracy_score, confusion_matrix
def evaluate_model(model, X_flat, age, gender, max_age):
    age_pred_sc, g_pred = model.predict(X_flat)
    age_pred = age_pred_sc.squeeze()*max_age
    mae = mean_absolute_error(age.squeeze(), age_pred)
    acc = accuracy_score(np.argmax(gender,1), np.argmax(g_pred,1))
    cm  = confusion_matrix(np.argmax(gender,1), np.argmax(g_pred,1))
    return mae, acc, cm


In [8]:
model = MultiTaskNN(input_dim=X_train_flat.shape[1], alpha=1.0, beta=5.0)

hist_loss, hist_mae, hist_acc = train_model(
    model, X_train_flat, age_train_scaled, gender_train,
    max_age=MAX_AGE, epochs=25, batch=128, lr=1e-4
)

mae, acc, cm = evaluate_model(model, X_test_flat, age_test, gender_test, MAX_AGE)
print(f"\n📊 Test MAE: {mae:.2f} years | Gender Acc: {acc*100:.2f}%")
print("Confusion matrix:\n", cm)


Epoch 01 | Loss 4.0749 | MAE 60.89 | Acc 52.93%
Epoch 02 | Loss 3.7821 | MAE 46.61 | Acc 53.77%
Epoch 03 | Loss 3.6676 | MAE 41.04 | Acc 54.32%
Epoch 04 | Loss 3.6211 | MAE 38.79 | Acc 54.71%
Epoch 05 | Loss 3.6002 | MAE 37.71 | Acc 54.99%
Epoch 06 | Loss 3.5888 | MAE 37.10 | Acc 55.14%
Epoch 07 | Loss 3.5814 | MAE 36.68 | Acc 55.26%
Epoch 08 | Loss 3.5761 | MAE 36.34 | Acc 55.32%
Epoch 09 | Loss 3.5717 | MAE 36.03 | Acc 55.48%
Epoch 10 | Loss 3.5675 | MAE 35.76 | Acc 55.64%
Epoch 11 | Loss 3.5637 | MAE 35.49 | Acc 55.78%
Epoch 12 | Loss 3.5601 | MAE 35.23 | Acc 55.82%
Epoch 13 | Loss 3.5566 | MAE 34.99 | Acc 55.93%
Epoch 14 | Loss 3.5532 | MAE 34.76 | Acc 56.08%
Epoch 15 | Loss 3.5500 | MAE 34.53 | Acc 56.08%
Epoch 16 | Loss 3.5469 | MAE 34.31 | Acc 56.23%
Epoch 17 | Loss 3.5437 | MAE 34.10 | Acc 56.18%
Epoch 18 | Loss 3.5407 | MAE 33.89 | Acc 56.31%
Epoch 19 | Loss 3.5378 | MAE 33.69 | Acc 56.36%
Epoch 20 | Loss 3.5349 | MAE 33.50 | Acc 56.44%
Epoch 21 | Loss 3.5321 | MAE 33.32 | Acc